# API client for green fashion consumers to know animal ethical information of brands - web scrape data from public websites
website: https://goodonyou.eco/
The function provides a quick look at brandlists animal usage information, standards, and scores.
animal material ethical information include usage of fur, angora, down feather, shearling, karakul, exotic animal skin and hair, wool use including ‘mulesing’, and leather. Also check whether brands use these material align with standards.

skills:
Data Acquisition:
Import data from json.
Web scrape data from public websites.
Write an API client for an API and/or functions associated with API interaction.
Handle, parse, and transform JSON and XML.
Data Cleaning, Transformation, and Organization:
Use data wrangling to transform data into a dataframe ready for green consumers reference and analysis.
Use loops processes.
Use functions and functional programming to export repetitive or difficult tasks.
Handle and process strings and use regular expressions.
Documentation and Presentation:
Provide and document useful functions as part of a Python package.
capture error
Use Jupyter Notebook to generate a vignette on how to use your package functions.

# create vignettes - presentation demo (class of popular fashion brand)
I retrieve 6 popoluar footwear and apparel brands' tickers yahoo finance: https://finance.yahoo.com/screener/88f2ce8e-1a19-4f03-ad8f-95e8b2c9c0d1?offset=25&=25
create a class of brandbook for later presentation of features

In [232]:
class trendbrand():
    def __init__(self, name='', btype=''):
        self.name = name
        self.type = btype
    def __str__(self):
        return f"brand name: {self.name}; brand type: {self.type}\n"
    
b1 = trendbrand(name='NIKE', btype = 'footwear')
b2 = trendbrand(name='adidas', btype = 'footwear')
b3 = trendbrand(name='ASICS', btype = 'footwear')
b4 = trendbrand(name='cos', btype = 'apparel')
b5 = trendbrand(name='theory', btype = 'apparel')
b6 = trendbrand(name='lululemon', btype = 'apparel')

class Brandbook:
    def __init__(self):
        self.bb = []

    def add(self, trendbrand):
        self.bb.append(trendbrand)
        
    def __str__(self):
        temp = ""
        for fasbrand in self.bb:
            temp = temp + str(fasbrand.name) + "\n"
        return temp
    
    def get_type(self, brand_type):
        gettype=[]
        for fasbrand in self.bb:
            if brand_type==fasbrand.type:
                gettype.append(fasbrand.name)
        return gettype


#make brand book
bbook = Brandbook()
#add brands
b_list = [b1, b2, b3, b4, b5, b6]
for b in b_list:
    bbook.add(b)

In [233]:
def brandbag(brand_type):  
    return bbook.get_type(brand_type=brand_type)


In [235]:
footwearvig = brandbag('footwear')
apparelvig = brandbag('apparel')


['cos', 'theory', 'lululemon']

## Web scrape data from public websites: good on you.
+ presentation: brandbook


In [221]:
import re
import lxml
from lxml import html
import requests
from bs4 import BeautifulSoup
import pandas as pd

def brandaniethical(brandlist):
    
    """
    Get fashion brands' animal ethical usage information. provide information of whether the brand use fur, angora, down feather, shearling, karakul, exotic animal skin and hair, wool use including ‘mulesing’, and leather 
    Also check whether brands use these material align with standards.
    
    Parameters
    ----------
    brandlist : list
      A list of brands name that consumers want to know their condition of animal usage
    
    Returns
    -------
    pandas.dataframe
      The dataframe of brands ethical consumption of animal material which include usage of fur, angora, down feather, shearling, karakul, exotic animal skin and hair, wool use including ‘mulesing’, and leather. Also check whether brands use these material align with standards.
      
    Examples
    --------
    >>> from brandaniethical import brandaniethical 
    >>> brandaniethical.brandaniethical(['cos','theory','lululemon','nike', 'skechers'])
	company	animal_score	update	fur	angora	leather	wool	shearling	karakul	down	feather	exotic animal hair	exotic animal skin	down_RDS	wool_mulesing
0	cos	3/5	July 2020	0	0	1	1	0	0	1	0	1	0	1	1
1	theory	2/5	December 2019	0	0	1	1	0	0	1	0	1	0	0	0
2	lululemon	2/5	July 2020	0	0	1	1	0	0	1	0	1	0	1	0
3	nike	2/5	July 2020	0	0	1	1	0	0	1	0	1	0	0	0
4	skechers	2/5	August 2020	0	0	1	1	0	0	0	0	0	0	0	0
    """
    assert isinstance(brandlist, list), f"This function only works on list but brandlist={brandlist}."
    for b in brandlist:
        assert isinstance(b, str), f"This function only works on string but brand={b}."
    brandlist = [b.lower() for b in brandlist]
    ani_reg = '[^.]* animal [^.]*\.'
    use_reg = '[^.]* (uses) [^.]*\.'
    date_reg = '(Last Updated: )(\w*\s\d...)'
    web = 'https://directory.goodonyou.eco/'
    animal_score = []
    ani_rate= []
    company = []
    update = []
    def brandani(brandlist):
        df_gfy=pd.DataFrame()
        for brand in brandlist:
            goypage = requests.get(web +'brand/'+ brand)
            soup = BeautifulSoup(goypage.content, 'html.parser')
            try:
                animal_score.append(str(soup.find_all("span", {"class": "StyledText-sc-1sadyjn-0 ccIhDL"})[2].string))
                ani_rate.append([string for string in soup.stripped_strings if re.search(ani_reg, string)][0])
                update.append([string for string in soup.stripped_strings if re.search(date_reg, string)][0]) 
            except (IndexError, TypeError, NameError, KeyError):
                print(f'please check company name of {brand}')
                pass
            else:
                df_gfy['company'] = pd.Series(brandlist)
                df_gfy['animal_score'] = pd.Series(animal_score)
                df_gfy['ani_rate'] = pd.Series(ani_rate)
                df_gfy['update'] = pd.Series(update)
                pd.set_option('display.max_colwidth', None)
        return df_gfy
    global dfydf
    dfydf = brandani(brandlist)
    #check animal material usage
    #data wrangling
    try:
        
        dfydf['ani_material'] = [re.search(use_reg, i).group(0) for i in dfydf['ani_rate']]

    except (IndexError, TypeError, NameError, KeyError):
        print(f'no animal material information')
        

    else:
        dfydf['update'] = [re.search('\w*\s\d...', i).group(0) for i in dfydf['update']]
        dfydf['animal_score'] = [re.sub(' out of ', '/', i) for i in dfydf['animal_score']]
        fabric = ['fur','angora', 'leather', 'wool', 'shearling', 'karakul', 'down', 'feather', 'exotic animal hair', 'exotic animal skin']
        for f in fabric:
            dfydf[f]=dfydf['ani_material'].str.contains(f)
        #check if use down whether down accredited by the Responsible Down Standar
        RDS_reg = 'It uses down accredited by the Responsible Down Standar'
        dfydf['down_RDS'] = dfydf['ani_rate'].str.contains(RDS_reg)
        dfydf.loc[dfydf['down_RDS']==True,'down'] = True
        #check if use wool whether wool includes mulesing
        dfydf['wool_mulesing'] = dfydf['ani_rate'].str.contains('mules')
        dfydf.loc[dfydf['wool_mulesing']==True,'wool'] = True
        df_aniusage = dfydf.loc[:,dfydf.columns!='ani_material']
        df_material = df_aniusage.replace(True,1)
        df_material = df_material.replace(False,0)
        
        return df_material.loc[:,df_material.columns!='ani_rate']
       

In [222]:
brandaniethical(['cos','theory','lululemon','nike', 'skechers'])

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,cos,3/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,1
1,theory,2/5,December 2019,0,0,1,1,0,0,1,0,1,0,0,0
2,lululemon,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,0
3,nike,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,0,0
4,skechers,2/5,August 2020,0,0,1,1,0,0,0,0,0,0,0,0


In [228]:
brandaniethical(footwearvig)

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,nike,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,0,0
1,adidas,3/5,July 2020,0,0,1,1,0,0,0,0,1,0,0,1
2,asics,2/5,August 2020,0,0,1,1,0,0,1,0,1,0,0,1


In [236]:
brandaniethical(apparelvig)

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,cos,3/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,1
1,theory,2/5,December 2019,0,0,1,1,0,0,1,0,1,0,0,0
2,lululemon,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,0
